In [ ]:
# params.yaml
'''
model_training:
  epochs: 20
  batch_size: 32
  lstm_units_1: 64
  lstm_units_2: 64
  dense_units_1: 128
  dropout_rate: 0.5
'''

In [ ]:
# config/config.yaml
'''
model_training:
  root_dir: artifacts/model_training
  trained_model_name: lstm_model.keras
'''

In [ ]:
## finance_ml.entity.config_entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_name: str

@dataclass(frozen=True)
class ModelTrainingParams: # Add a data class for model parameters
    epochs: int
    batch_size: int
    lstm_units_1: int
    lstm_units_2: int
    dense_units_1: int
    dropout_rate: float

In [ ]:
# src/finance_ml/config/configuration.py

from finance_ml.constants import *
from finance_ml.utils.common import read_yaml, create_directories
from finance_ml.entity.config_entity import DataIngestionConfig, DataTransformationConfig, ModelTrainingConfig, ModelTrainingParams # Import ModelTrainingParams
import os

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> tuple[ModelTrainingConfig, ModelTrainingParams]: # Modify return type
        config = self.config.model_training
        params = self.params.model_training # Access parameters from params.yaml

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            trained_model_name=config.trained_model_name
        )

        model_training_params = ModelTrainingParams(
            epochs=params.epochs,
            batch_size=params.batch_size,
            lstm_units_1=params.lstm_units_1,
            lstm_units_2=params.lstm_units_2,
            dense_units_1=params.dense_units_1,
            dropout_rate=params.dropout_rate
        )

        return model_training_config, model_training_params 

In [ ]:
# src/finance_ml/component/model_training.py

from finance_ml.entity.config_entity import ModelTrainingConfig, ModelTrainingParams # Import ModelTrainingParams
from tensorflow import keras
import numpy as np
import os
import tensorflow as tf

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig, params: ModelTrainingParams): # Accept both
        self.config = config
        self.params = params

    def train_model(self):
        # Load the transformed data
        X_train = np.load(os.path.join("artifacts", "data_transformation", "X_train.npy"))
        y_train = np.load(os.path.join("artifacts", "data_transformation", "y_train.npy"))
        X_test = np.load(os.path.join("artifacts", "data_transformation", "X_test.npy"))
        y_test = np.load(os.path.join("artifacts", "data_transformation", "y_test.npy"))

        # Build the Model using parameters
        model = keras.models.Sequential()

        model.add(keras.layers.LSTM(self.params.lstm_units_1, return_sequences=True, input_shape=(X_train.shape[1], 1)))
        model.add(keras.layers.LSTM(self.params.lstm_units_2, return_sequences=False))
        model.add(keras.layers.Dense(self.params.dense_units_1, activation="relu"))
        model.add(keras.layers.Dropout(self.params.dropout_rate))
        model.add(keras.layers.Dense(1))

        model.compile(optimizer="adam",
                      loss="mae",
                      metrics=[keras.metrics.RootMeanSquaredError()])

        # Train the model using parameters
        model.fit(X_train, y_train, epochs=self.params.epochs, batch_size=self.params.batch_size)

        # Save the trained model
        model.save(os.path.join(self.config.root_dir, self.config.trained_model_name))

        return model


In [ ]:
# src/finance_ml/pipeline/stage_03_model_training.py

from src.finance_ml.config.configuration import ConfigurationManager
# from src.finance_ml.component.model_training import 
ModelTraining
from src.finance_ml import logger
import pandas as pd
import os

STAGE_NAME = "Model Training stage"

class ModelTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        try:
            # Get configuration and parameters
            config_manager = ConfigurationManager()
            model_training_config, model_training_params = config_manager.get_model_training_config()

            # Perform model training with both config and params
            model_training = ModelTraining(config=model_training_config, params=model_training_params)
            model_training.train_model()

            logger.info("Model training completed successfully.")

        except Exception as e:
            logger.exception(e)
            raise e

# Example of how to run this stage
if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = ModelTrainingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e
